In [52]:
import pennylane as qml
from pennylane import numpy as np

#number of qubits and data dimension
n_qubits = 3  
dev = qml.device("default.qubit", wires=n_qubits)

#Step 1: Generate intermediate quantum state
def generate_intermediate_state(x, w):
    norm_x = np.linalg.norm(x)
    x = x / norm_x if norm_x != 0 else x
    for i, val in enumerate(x):
        qml.RX(val, wires=i)
        qml.RZ(val, wires=i)  #feature encoding
    for i, val in enumerate(w):
        qml.RY(val, wires=i)
    qml.CZ(wires=[0, 1])
    qml.CZ(wires=[1, 2])  #entanglement


#step 2: Quantum amplitude estimation for gradient computation
@qml.qnode(dev)
def compute_gradient_amplitude(x, weights):
    quantum_feature_map(x)
    for i in range(len(weights)):
        qml.RY(weights[i], wires=i)  # First layer of learnable parameters
    qml.CZ(wires=[0, 1])  #entanglement
    for i in range(len(weights)):
        qml.RY(weights[i] / 2, wires=i)  # Second layer of learnable parameters
    return qml.probs(wires=range(len(x)))

def quantum_feature_map(x):
    norm_x = np.linalg.norm(x)
    x = x / norm_x if norm_x != 0 else x
    for i, val in enumerate(x):
        qml.RX(val, wires=i)
        qml.RZ(val, wires=i)  # Add non-linear feature encoding
    qml.CZ(wires=[0, 1])  # Add entanglement
    qml.CZ(wires=[1, 2])  # Additional entanglement


#step 3: Classical gradient computation
def classical_gradient(x, y, w):
    """
    Computes the classical gradient using quantum measurements.
        x: Input data point
        y: Ground truth label for the data point
        w: Parameters
    """
    amplitude_probs = compute_gradient_amplitude(x, w)
    amplitude_probs = np.clip(amplitude_probs, 1e-6, 1 - 1e-6)  #clip probabilities
    gradient = np.zeros_like(w)
    for i in range(len(w)):
        gradient[i] = (2 * amplitude_probs[i] - 1) * np.linalg.norm(x)
    return gradient

#training Loop
def train_quantum_logistic_regression(X, y, w_init, lr, epsilon, max_steps):
    w = w_init
    for step in range(max_steps):
        grad = np.zeros_like(w)
        for i, x in enumerate(X):
            gradient = classical_gradient(x, y[i], w)
            grad += gradient
        grad /= len(X)

        #clip gradients for updates
        grad = np.clip(grad, -1, 1)
        
        w = w - lr * grad
        if np.linalg.norm(grad) < epsilon:
            print(f"Converged after {step} steps.")
            break
    return w

In [59]:
# #regularized Logistic Loss Function

def cost(weights, X, y):
    predictions = np.array([compute_gradient_amplitude(x, weights)[0] for x in X])
    predictions = np.clip(predictions, 1e-6, 1 - 1e-6) 
    loss = np.mean(-y * np.log(predictions) - (1 - y) * np.log(1 - predictions))
    regularization = 0.001 * np.sum(weights**2)  
    return loss + regularization


#gradient Descent with Cost Function Integration
def train_quantum_logistic_regression_with_cost(X, y, w_init, lr, epsilon, max_steps):
    """
    Train the quantum logistic regression model using the cost function.
    """
    w = w_init
    for step in range(max_steps):
        grad = np.zeros_like(w)
        for i, x in enumerate(X):
            gradient = classical_gradient(x, y[i], w)
            grad += gradient
        grad /= len(X)
        
        # Compute the current cost for monitoring
        current_cost = cost(w, X, y)
        print(f"Step {step}, Cost: {current_cost}")
        
        # Update weights using gradient descent
        grad = np.clip(grad, -1, 1)  # Gradient clipping for stability
        w = w - lr * grad
        
        # Check for convergence
        if np.linalg.norm(grad) < epsilon:
            print(f"Converged after {step} steps.")
            break
    
    return w


In [61]:
#normalize
X = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1]])
X_norm = np.linalg.norm(X, axis=1, keepdims=True)
X_norm[X_norm == 0] = 1  # Prevent division by zero
X = X / X_norm

#initialize weights
w_init = np.random.uniform(-0.1, 0.1, size=3)  # Small initial weights

#train
w_optimal = train_quantum_logistic_regression_with_cost(X, y, w_init, lr=0.001, epsilon=1e-6, max_steps=1000)
print("Optimized weights:", w_optimal)

Step 0, Cost: 1.6522732954608972
Step 1, Cost: 1.6487358027192571
Step 2, Cost: 1.6452179624750178
Step 3, Cost: 1.6417196477209688
Step 4, Cost: 1.6382407302779491
Step 5, Cost: 1.6347810809439318
Step 6, Cost: 1.6313405696344474
Step 7, Cost: 1.6279190655146103
Step 8, Cost: 1.624516437123205
Step 9, Cost: 1.6211325524894
Step 10, Cost: 1.617767279242349
Step 11, Cost: 1.6144204847140353
Step 12, Cost: 1.611092036035932
Step 13, Cost: 1.6077818002295514
Step 14, Cost: 1.6044896442914307
Step 15, Cost: 1.6012154352725467
Step 16, Cost: 1.5979590403529598
Step 17, Cost: 1.5947203269113954
Step 18, Cost: 1.5914991625904338
Step 19, Cost: 1.5882954153573394
Step 20, Cost: 1.5851089535608358
Step 21, Cost: 1.5819396459841037
Step 22, Cost: 1.5787873618940833
Step 23, Cost: 1.575651971087344
Step 24, Cost: 1.5725333439327573
Step 25, Cost: 1.5694313514110674
Step 26, Cost: 1.566345865151615
Step 27, Cost: 1.5632767574662711
Step 28, Cost: 1.56022390138079
Step 29, Cost: 1.5571871706637632


In [62]:
print("Input Data (X):", X)
print("Labels (y):", y)

Input Data (X): [[0.         0.         0.        ]
 [0.         1.         0.        ]
 [1.         0.         0.        ]
 [0.57735027 0.57735027 0.57735027]]
Labels (y): [0 1 1 0]


In [63]:
def predict(x, w):
    """Make predictions using the trained weights."""
    norm_x = np.linalg.norm(x)
    x = x / norm_x if norm_x != 0 else x  
    prob = compute_gradient_amplitude(x, w)[0]  
    return 1 if prob >= 0.5 else 0

predictions = [predict(x, w_optimal) for x in X]
print("Predictions:", predictions)
print("Ground Truth:", y)


Predictions: [1, 0, 1, 0]
Ground Truth: [0 1 1 0]


In [64]:
accuracy = np.mean([predict(x, w_optimal) == label for x, label in zip(X, y)])
print(f"Training Accuracy: {accuracy * 100:.2f}%")

Training Accuracy: 50.00%


In [84]:
# X = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1]])
# y = np.array([0, 1, 1, 0])  
X = np.array([
    [0, 0, 0],
    [0, 1, 0],
    [1, 0, 0],
    [1, 1, 1],
    [0, 0, 1],
    [1, 0, 1],
    [0, 1, 1]
])
y = np.array([0, 1, 1, 0, 1, 0, 1])  
X_norm = np.linalg.norm(X, axis=1, keepdims=True)
X_norm[X_norm == 0] = 1  
X = X / X_norm


In [85]:
import pennylane as qml
from pennylane import numpy as np

n_qubits = 3
dev = qml.device("default.qubit", wires=n_qubits)

def feature_map(x):
    for i in range(len(x)):
        qml.RX(np.pi * x[i], wires=i)
    qml.CZ(wires=[0, 1])
    qml.CZ(wires=[1, 2])

def variational_circuit(params):
    num_layers = 3
    for l in range(num_layers):
        for i in range(n_qubits):
            qml.RY(params[l, i], wires=i)
            qml.RZ(params[l, i + n_qubits], wires=i)
        for i in range(n_qubits):
            qml.CNOT(wires=[i, (i + 1) % n_qubits])

@qml.qnode(dev)
def circuit(x, params):
    feature_map(x)
    variational_circuit(params)
    return qml.expval(qml.PauliZ(0))


In [87]:
def cost(params, X, y):
    loss = 0
    for xi, yi in zip(X, y):
        pred = (circuit(xi, params) + 1) / 2  # Map from [-1, 1] to [0, 1]
        loss += (pred - yi) ** 2
    return loss / len(X)
params = np.random.uniform(0, 2 * np.pi, size=(2 * n_qubits,))

opt = qml.GradientDescentOptimizer(stepsize=0.1)

max_steps = 500
for step in range(max_steps):
    params = opt.step(lambda p: cost(p, X, y), params)
    if step % 10 == 0:
        current_cost = cost(params, X, y)
        print(f"Step {step}, Cost: {current_cost}")


TypeError: loop of ufunc does not support argument 0 of type ArrayBox which has no callable log method

In [88]:
for xi, yi in zip(X, y):
    pred = (circuit(xi, params) + 1) / 2
    print(f"Input: {xi}, Predicted: {pred:.3f}, Actual: {yi}")


Input: [0. 0. 0.], Predicted: 0.341, Actual: 0
Input: [0. 1. 0.], Predicted: 0.656, Actual: 1
Input: [1. 0. 0.], Predicted: 0.623, Actual: 1
Input: [0.57735027 0.57735027 0.57735027], Predicted: 0.272, Actual: 0
Input: [0. 0. 1.], Predicted: 0.645, Actual: 1
Input: [0.70710678 0.         0.70710678], Predicted: 0.629, Actual: 0
Input: [0.         0.70710678 0.70710678], Predicted: 0.441, Actual: 1


In [90]:
import pennylane as qml
from pennylane import numpy as np  

n_qubits = 3
dev = qml.device("default.qubit", wires=n_qubits)

def feature_map(x):
    for i in range(len(x)):
        qml.RX(np.pi * x[i], wires=i)
        qml.RZ(np.pi * x[i], wires=i)
    for i in range(n_qubits):
        qml.CNOT(wires=[i, (i + 1) % n_qubits])

def variational_circuit(params):
    num_layers = params.shape[0]
    for l in range(num_layers):
        for i in range(n_qubits):
            qml.RY(params[l, i], wires=i)
            qml.RZ(params[l, i + n_qubits], wires=i)
        for i in range(n_qubits):
            qml.CNOT(wires=[i, (i + 1) % n_qubits])

@qml.qnode(dev, interface='autograd')
def circuit(x, params):
    feature_map(x)
    variational_circuit(params)
    return qml.expval(qml.PauliZ(0))

def cost(params, X, y):
    predictions = [(circuit(xi, params) + 1) / 2 for xi in X]
    predictions = qml.numpy.stack(predictions)
    predictions = qml.numpy.clip(predictions, 1e-6, 1 - 1e-6)
    y = qml.numpy.array(y)
    loss = -qml.numpy.mean(y * qml.numpy.log(predictions) + (1 - y) * qml.numpy.log(1 - predictions))
    return loss

# Input 
X = qml.numpy.array([
    [0, 0, 0], 
    [0, 1, 0],
    [1, 0, 0],
    [1, 1, 1],
    [0, 0, 1],
    [1, 0, 1],
    [0, 1, 1]
])

y = qml.numpy.array([0, 1, 1, 0, 1, 0, 1])  

#normalize 
X_norm = qml.numpy.linalg.norm(X, axis=1, keepdims=True)
X_norm[X_norm == 0] = 1  
X = X / X_norm

#initialize 
num_layers = 3
params = qml.numpy.random.uniform(0, 2 * qml.numpy.pi, size=(num_layers, 2 * n_qubits))

#set up an Adam optimizer with a smaller learning rate
opt = qml.AdamOptimizer(stepsize=0.01)

#training Loop
max_steps = 1000
for step in range(max_steps):
    params = opt.step(lambda p: cost(p, X, y), params)
    if step % 50 == 0:
        current_cost = cost(params, X, y)
        print(f"Step {step}, Cost: {current_cost}")

#testing the Model
print("\nTesting the Model:")
for xi, yi in zip(X, y):
    pred = (circuit(xi, params) + 1) / 2
    predicted_label = int(pred > 0.5)
    print(f"Input: {xi}, Predicted Probability: {pred:.3f}, Predicted Label: {predicted_label}, Actual: {yi}")


Step 0, Cost: 0.58018815930157
Step 50, Cost: 0.3391633350827611
Step 100, Cost: 0.314622631512258
Step 150, Cost: 0.2623522719496512
Step 200, Cost: 0.208048122768597
Step 250, Cost: 0.17490883080219294
Step 300, Cost: 0.17418483774884377
Step 350, Cost: 0.17416668702107216
Step 400, Cost: 0.17416652160529233
Step 450, Cost: 0.17416652133375382
Step 500, Cost: 0.17416652133330554
Step 550, Cost: 0.17416652133330485
Step 600, Cost: 0.17416652133330474
Step 650, Cost: 0.17416652133330487
Step 700, Cost: 0.17416652133330482
Step 750, Cost: 0.17416652133330487
Step 800, Cost: 0.17416652133330454
Step 850, Cost: 0.17416666260040528
Step 900, Cost: 0.17416652392601967
Step 950, Cost: 0.17416652136383926

Testing the Model:
Input: [0. 0. 0.], Predicted Probability: 0.069, Predicted Label: 0, Actual: 0
Input: [0. 1. 0.], Predicted Probability: 0.873, Predicted Label: 1, Actual: 1
Input: [1. 0. 0.], Predicted Probability: 0.914, Predicted Label: 1, Actual: 1
Input: [0.57735027 0.57735027 0.577

A part of the dataset

In [94]:
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

data = pd.read_csv('PS_20174392719_1491204439457_log.csv')

#subset of the data for computational feasibility
data = data.sample(n=1000, random_state=42)

features = data[['amount', 'oldbalanceOrg']].values
labels = data['isFraud'].values.astype(float)  # Convert labels to float

#normalize 
scaler = StandardScaler()
features = scaler.fit_transform(features)

features = np.array(features)
labels = np.array(labels)

#quantum circuit parameters
n_qubits = features.shape[1]
dev = qml.device('default.qubit', wires=n_qubits)

def feature_map(x):
    for i in range(len(x)):
        qml.RX(x[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])

def variational_circuit(params):
    for i in range(n_qubits):
        qml.RY(params[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

@qml.qnode(dev, interface='autograd')
def circuit(x, params):
    feature_map(x)
    variational_circuit(params)
    return qml.expval(qml.PauliZ(0))

def cost(params, X, y):
    y = np.array(y)  
    predictions = [(circuit(xi, params) + 1) / 2 for xi in X]
    predictions = np.stack(predictions)  
    predictions = np.clip(predictions, 1e-6, 1 - 1e-6)  
    loss = -np.mean(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))
    return loss


params = np.random.uniform(0, 2 * np.pi, size=(n_qubits,))

opt = qml.AdamOptimizer(stepsize=0.05)

#training loop
max_steps = 50
for step in range(max_steps):
    params = opt.step(lambda p: cost(p, features, labels), params)
    if step % 10 == 0:
        current_cost = cost(params, features, labels)
        print(f"Step {step}, Cost: {current_cost}")

#evaluate the model
predictions = [(circuit(xi, params) + 1) / 2 for xi in features]
predicted_labels = (np.array(predictions) > 0.5).astype(int)

#calculate accuracy
accuracy = accuracy_score(labels, predicted_labels)
print(f"\nModel accuracy: {accuracy}")


Step 0, Cost: 0.6932500939008935
Step 10, Cost: 0.35715124203645565
Step 20, Cost: 0.18505877173700128
Step 30, Cost: 0.12112138911981549
Step 40, Cost: 0.10966481788669509

Model accuracy: 0.973


In [97]:
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

data = pd.read_csv('PS_20174392719_1491204439457_log.csv')
data = data.sample(n=10000, random_state=42)

data = data.drop(['oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest'], axis=1)

type_encoder = OneHotEncoder(sparse_output=False)
type_encoded = type_encoder.fit_transform(data[['type']])

features = np.hstack((
    data[['amount', 'isFlaggedFraud']].values,
    type_encoded
))
labels = data['isFraud'].values.astype(float)  
scaler = StandardScaler()
features = scaler.fit_transform(features)

features = np.array(features)
labels = np.array(labels)

n_qubits = features.shape[1]
dev = qml.device('default.qubit', wires=n_qubits)

def feature_map(x):
    for i in range(len(x)):
        qml.RX(x[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])

def variational_circuit(params):
    for i in range(n_qubits):
        qml.RY(params[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

@qml.qnode(dev, interface='autograd')
def circuit(x, params):
    feature_map(x)
    variational_circuit(params)
    return qml.expval(qml.PauliZ(0))

def cost(params, X, y):
    y = np.array(y)  
    predictions = [(circuit(xi, params) + 1) / 2 for xi in X]
    predictions = np.stack(predictions)  
    predictions = np.clip(predictions, 1e-6, 1 - 1e-6)  
    loss = -np.mean(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))
    return loss

params = np.random.uniform(0, 2 * np.pi, size=(n_qubits,))

opt = qml.AdamOptimizer(stepsize=0.05)

max_steps = 50
for step in range(max_steps):
    params = opt.step(lambda p: cost(p, features, labels), params)
    if step % 10 == 0:
        current_cost = cost(params, features, labels)
        print(f"Step {step}, Cost: {current_cost}")

predictions = [(circuit(xi, params) + 1) / 2 for xi in features]
predicted_labels = (np.array(predictions) > 0.5).astype(int)

accuracy = accuracy_score(labels, predicted_labels)
print(f"\nModel accuracy: {accuracy}")


Step 0, Cost: 0.5064614980225101
Step 10, Cost: 0.23389472111059076
Step 20, Cost: 0.11067603649221922
Step 30, Cost: 0.07888306056502507
Step 40, Cost: 0.07795173209772441

Model accuracy: 0.9792


In [98]:
current_cost

tensor(0.07795173, requires_grad=True)

In [100]:
labels

tensor([0., 0., 0., ..., 0., 0., 0.], requires_grad=True)

In [102]:
predicted_labels

tensor([0, 0, 0, ..., 0, 0, 0], requires_grad=True)

In [103]:
params

tensor([3.21684702, 4.15582199, 3.33690872, 3.38518135, 4.08758266,
        2.44025071, 3.3761743 ], requires_grad=True)

In [104]:
features

tensor([[ 0.33572084, -0.0100005 ,  1.86986707, ..., -0.07637955,
         -0.71502369, -0.30184015],
        [-0.34763888, -0.0100005 , -0.53479737, ..., -0.07637955,
          1.39855506, -0.30184015],
        [-0.04600445, -0.0100005 ,  1.86986707, ..., -0.07637955,
         -0.71502369, -0.30184015],
        ...,
        [-0.13130636, -0.0100005 , -0.53479737, ..., -0.07637955,
         -0.71502369, -0.30184015],
        [-0.37142702, -0.0100005 , -0.53479737, ..., -0.07637955,
          1.39855506, -0.30184015],
        [-0.16550449, -0.0100005 ,  1.86986707, ..., -0.07637955,
         -0.71502369, -0.30184015]], requires_grad=True)

In [111]:
type_encoded[:5]

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.]])

In [108]:
data['type'].unique()

array(['CASH_IN', 'PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT'],
      dtype=object)

In [109]:
data

,step,type,amount,nameOrig,nameDest,isFraud,isFlaggedFraud
3737323,278,CASH_IN,330218.42,C632336343,C834976624,0,0
264914,15,PAYMENT,11647.08,C1264712553,M215391829,0,0
85647,10,CASH_IN,152264.21,C1746846248,C1607284477,0,0
5899326,403,TRANSFER,1551760.63,C333676753,C1564353608,0,0
2544263,206,CASH_IN,78172.30,C813403091,C1091768874,0,0
...,...,...,...,...,...,...,...
4094229,301,CASH_OUT,80646.44,C819258095,C1433007346,0,0
5603252,394,CASH_IN,57501.43,C1316678628,C483888900,0,0
6189570,570,CASH_OUT,112497.83,C765932428,C355959262,0,0
5853779,402,PAYMENT,557.43,C2108927530,M918034835,0,0


In [110]:
'''
CASH_IN -> 1 0 0 0 0
PAYMENT -> 0 0 0 1 0
CASH_OUT-> 0 1 0 0 0
TRANSFER-> 0 0 0 0 1
DEBIT ->   0 0 1 0 0 
'''

'\nCASH_IN -> 1 0 0 0 0\nPAYMENT -> 0 0 0 1 0\nCASH_OUT-> 0 1 0 0 0\n'

In [112]:
import numpy as np

np.save('trained_params_trial1.npy', params)

print("Model parameters have been saved to 'trained_params.npy'")


Model parameters have been saved to 'trained_params.npy'


In [114]:
import pickle


with open('scaler_trial1.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('type_encoder_trial1.pkl', 'wb') as f:
    pickle.dump(type_encoder, f)

print("Scaler and encoder have been saved.")


Scaler and encoder have been saved.


In [116]:
predictions = [quantum_decision_tree_predict(x, params) for x in X_test]
predictions = np.array(predictions)

accuracy = accuracy_score(y_test, predictions)
print(f"\nTest set accuracy: {accuracy:.2f}")

print("\nClassification Report:")
print(classification_report(y_test, predictions))

cm = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(cm)


NameError: name 'X_test' is not defined